# Bigraph-Builder Demo

In [4]:
from biosimulator_processes.biosimulator_builder import BiosimulatorBuilder 
from builder import ProcessTypes

## Initialize the builder

In [2]:
b = BiosimulatorBuilder()

In [3]:
print(b.list_processes())

['CopasiProcess', 'TelluriumProcess', 'DeterministicTimeCourseParameterScan', 'console-emitter', 'ram-emitter', 'SmoldynProcess', 'CobraProcess']


In [25]:
import ast
from typing import Dict, Any 
from builder import Builder


def generate_input_kwargs() -> Dict[str, Any]:
    """Generate kwargs to be used as dynamic input for process configuration.
    
        Args:
            None.
        Returns:
            Dict[str, Any]: configuration kwargs for process construction.
    """
    process_kwargs = input('Please enter the process configuration keyword arguments: ')
    process_args = process_kwargs.split(',')
    input_kwargs = {}
    for arg in process_args:
        key, value = arg.split('=')
        try:
            # safely evaluate the value to its actual data type
            input_kwargs[key.strip()] = ast.literal_eval(value)
        except (ValueError, SyntaxError):
            input_kwargs[key] = value
    return input_kwargs


def add_single_process(b: Builder):
    process_type = input(f'Please enter one of the following process types that you wish to add:\n{b.list_processes()}\n:')
    builder_node_name = input('Please enter the name that you wish to assign to this process: ')
    input_kwargs = generate_input_kwargs()
    visualize = input('Do you wish to visualize this addition after (y/N): ')
    b.add_process(process_id=builder_node_name, name=process_type, config={**input_kwargs})
    b.connect_all()
    if 'N' in visualize:
        b.visualize()
    

In [33]:
generate_input_kwargs()

{'model': {'model_source': '/users/alex'}, 'b': 34.2342}

In [11]:
add_single_process()

TypeError: string indices must be integers

In [ ]:
b['event_process'].add_process(
    name='GillespieEvent',
    kdeg=1.0,  # kwargs fill parameters in the config
)

In [ ]:
# visualize shows the process with its disconnected ports
b.visualize()

### print ports

In [ ]:
b['event_process'].interface(True)

### connect ports using connect_all
`Builder.connect_all` connects ports to stores of the same name.

In [ ]:
b.connect_all(append_to_store_name='_store')
b

In [ ]:
b.visualize()

### add interval process to the config

In [ ]:
b['interval_process'].add_process(
    name='GillespieInterval',
)

In [ ]:
b.visualize()

### connect port to specific target

In [ ]:
# to connect a port in a more targeted way, use connect and specify the port and its target path
b['interval_process'].connect(port='interval', target=['event_process', 'interval']) 

# the remaining ports can connect_all
b.connect_all() 

In [ ]:
b.visualize()

### check current Builder config

In [ ]:
b

## Update the initial state

In [ ]:
initial_state = {
    'DNA_store': {
        'A gene': 2.0,
        'B gene': 1.0},
}
b.update(initial_state)

## Generate composite from builder config and simulate

In [ ]:
composite = b.generate()
composite.run(10)

In [ ]:
composite

## Retrieve the composite document

In [ ]:
doc = b.document()
doc

### save the document to file

In [ ]:
b.write(filename='toy_bigraph')

### load a composite from document
This document represents the full state of the composite, and so can reproduce the previous composite when loaded into a fresh Builder

In [ ]:
b2 = Builder(core=core, file_path='out/toy_bigraph.json')
b2

In [ ]:
b2.visualize()